# Introduction

In order to obtain information about the structure and composition of a planetary atmosphere from its electromagnetic spectrum, apart from the radiative transfer model, which relates the atmospheric parameters to the modelled spectrum, we need a retrieval tool to solve the inverse problem, which allows us to estimate what atmospheric parameters provide a best fit between the modelled and measured spectrum. There are several codes and techniques that can be applied to solve the inverse problem, each with its own advantages and disadvantages. In NEMESIS, two different retrieval methodologies can be applied: Optimal Estimation and Nested Sampling. 

<img src="../images/archNEMESIS_retrieval.png" alt="Drawing" style="width: 800px;"/>

# Optimal Estimation

## Description of the algorithm

**NOTE**: A complete description of the implementation of the Optimal Estimation framework in NEMESIS is given in [Irwin et al. (2008)](https://doi.org/10.1016/j.jqsrt.2007.11.006).  

NEMESIS uses the non-linear optimal estimator formalism [(Rodgers, 2000)](https://www.worldscientific.com/worldscibooks/10.1142/3171), which iteratively minimises the difference between the modelled and measured spectra, subject to a minimum departure from an *a priori* state of the atmosphere, by minimising the cost function

\begin{equation}
   \phi = (\mathbf{y}_m - \mathbf{y}_n)^T \mathbf{S}_{\epsilon}^{-1} (\mathbf{y}_m - \mathbf{y}_n) + (\mathbf{x}_n - \mathbf{x}_a)^T \mathbf{S}_a^{-1} (\mathbf{x}_n - \mathbf{x}_a),
\end{equation}

where $\mathbf{y}_m$ is the measured spectrum, $\mathbf{y}_n$ is the synthetic spectrum calculated using the forward model, $\mathbf{x}_n$ is the state vector, which includes the atmospheric parameters to be retrieved, $\mathbf{S}_{\epsilon}$ is the measurement covariance matrix, $\mathbf{x}_a$ is the *a priori* state vector and $\mathbf{S}_a$ is the *a priori* covariance matrix.

For atmospheric sounding in the Earth's atmosphere, climatology provides good statistics about the *a priori* state vector and covariance matrix. However, for planetary atmospheres no such statistical record exists, and in that case the diagonal elements of the *a priori* covariance matrix are set to the square of the estimated errors in the *a priori* state vector. In the case of retrieving continuous vertical profiles, the values at different altitudes are assumed to be correlated, and the off-diagonal terms of $\mathbf{S}_a$ are then set to follow

\begin{equation}
S_{ij} = \sqrt{S_{ii}S_{jj}} \cdot \exp\left(-\dfrac{|\ln{p_i/p_j}|}{L_c}\right)
\end{equation}

where $p_i$ represents the pressure at the *i*th level and $L_c$ is the correlation length, equivalent to the number of scale heights over which the continuous profile is assumed to be correlated.

The retrieval procedure NEMESIS applies to minimise the cost function is to calculate the state vector in the next iteration as

\begin{equation}
\mathbf{x}_{n+1} = \mathbf{x}_0 + \mathbf{S}_a \mathbf{K}_n^T (\mathbf{K}_n \mathbf{S}_a \mathbf{K}_n^T + \mathbf{S}_\epsilon)^{-1}(\mathbf{y}_m - \mathbf{y}_n - \mathbf{K}_n(\mathbf{x}_a - \mathbf{x}_n) )
\end{equation}

where $\mathbf{K}_n$ is the Jacobian matrix at the $n$th iteration, which represents the derivatives of  spectra at each wavelength with respect to each of the elements in the state vector. However, the convergence of the retrieval using this equation can be unstable due to the great variability of $\mathbf{K}_n$ over iterations. Instead, NEMESIS uses a scheme based on the Marquardt–Levenberg principal, in which the state vector at the next iteration is modified so that it follows

\begin{equation}
\mathbf{x}'_{n+1} = \mathbf{x}_n + \dfrac{\mathbf{x}_{n+1} - \mathbf{x}_n}{1 + \lambda},
\end{equation}

where $\lambda$ is initially set to 1. If the spectrum calculated using $\mathbf{x}'_{n+1}$ reduces the cost function, then $\mathbf{x}'_{n+1}$ is used as $\mathbf{x}_n$ in the next iteration, and $\lambda$ is multiplied by 0.3. In the opposite case, in which $\mathbf{x}'_{n+1}$ actually increases the cost function, then $\mathbf{x}_n$ is kept and the value of $\lambda$ is multiplied by 10. As the code converges, $\lambda \longrightarrow 0$ and $\mathbf{x}_n$ tends to the optimal estimate. 

The error in the optimal estimate is estimated to be the result of two main contributions. The first contribution is the smoothing error $\mathbf{S}_s$, which can be regarded as the effect in which the observing system smooths the profile. In such a case, the retrieval can be regarded as an estimate of the smoothed state rather than an estimate of the true state, or what is equivalent, an estimate of the true state, but with an error contribution due to smoothing. This contribution to the retrieved error can be calculated as:

\begin{equation}
\mathbf{S}_s = (\mathbf{K}^T \mathbf{S}_\epsilon^{-1} \mathbf{K} + \mathbf{S}_a^{-1})^{-1} \mathbf{S}_a^{-1} (\mathbf{K}^T \mathbf{S}_\epsilon^{-1} \mathbf{K} +  \mathbf{S}_a^{-1})^{-1}.
\end{equation}

The second contribution to the retrieved uncertainty comes from the propagation of the measurement error to the state vector. The covariance matrix of the retrieval noise can be calculated as

\begin{equation}
\mathbf{S}_m = (\mathbf{K}^T \mathbf{S}_\epsilon^{-1} \mathbf{K} + \mathbf{S}_a^{-1})^{-1} \mathbf{K}^T \mathbf{S}_\epsilon^{-1} \mathbf{K} (\mathbf{K}^T \mathbf{S}_\epsilon^{-1} \mathbf{K} + \mathbf{S}_a^{-1})^{-1}.
\end{equation}

In NEMESIS, the measurement error covariance matrix $\mathbf{S}_\epsilon$ is assumed diagonal, with each of the diagonal elements corresponding to the measured instrument noise equivalent transmission units. In addition, NEMESIS also allows the introduction of a forward modelling error (e.g., can be used to incorporate uncertainties in the line databases) to the measurement error covariance matrix, to set how well we think we can fit the spectra.

These two main contributions form the total retrieved uncertainty in the state vector, which is given by

\begin{equation}
\mathbf{S}_t = \mathbf{S}_m + \mathbf{S}_s =  (\mathbf{K}^T \mathbf{S}_\epsilon^{-1} \mathbf{K} + \mathbf{S}_a^{-1})^{-1}.
\end{equation}

## Optimal Estimation class

The Optimal Estimation implementation in NemesisPy is performed using a Python class. The parameters and attributes that fill this class are mostly the matrices relevant for this retrieval framework (e.g., Jacobian matrix, gain matrix, averaging kernels, state and measurement vectors, covariance matrices, etc.).

## Running an Optimal Estimation retrieval

Optimal Estimation is currently the default retrieval scheme in NemesisPy.

There are several parameters that need to be specified to run an Optimal Estimation retrieval:

- The *.apr* file includes the information about the *a priori* state vector $\mathbf{x}_a$ and the uncertainties in the state vector prior to the retrieval $\mathbf{S}_a$. The way in which the retrieval parameters and their uncertainties are specified depend on the model. A complete guide of the format for the different models is provided in the NEMESIS manual, and examples for the models included in NemesisPy are given in the Examples section of this website. Currently, there is no internal function in NemesisPy to write the *.apr* file, but it can be easily read using the *read_apr()* function of the Variables class.

- The maximum number of iterations (NITER) is specified in the *.inp* file. If NITER is set to -1, then the program will only run a forward model using the model parameterisations in the *.apr* file.

- The retrieval stops if a convergence criterion is satisfied: the reduction of the cost function $\phi$ is lower than a given factor (PHILIMIT). This factor is also specified in the *.inp* file.

Once the retrieval is finished, there are two outputs files that are generated:

- The *.mre* file includes the information about the best fit and the retrieved parameters. This file can be read using the *read_mre()* function.

- The *.cov* file includes information about the matrices relevant for the Optimal Estimation retrieval, such as the Jacobian matrix, averaging kernels, error covariance matrices, etc. This file can be read using the *read_cov()* function of the Optimal Estimation class.

# Nested Sampling

**NOTE**: This retrieval framework is actively being worked on. 


For background on nested sampling, see the Ultranest documention [here.](https://johannesbuchner.github.io/UltraNest/method.html) Note that Ultranest is not actually used in archNEMESIS (pymultinest is). 

For an example of how to set up a nested sampling run in archNEMESIS, see the Neptune_JWST_nested_sampling example notebook.